In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from scipy import optimize

In [2]:
import warnings
warnings.filterwarnings("ignore")  #самая важная и нужная вещь в этом ноутбуке!

**ЗАДАНИЕ 2**

In [3]:
def loglike(q, x):
    n = len(x)
    ll = n * np.log(2) - n * np.log(q) - (1 / q) * np.sum(np.square(x)) + np.sum(np.log(x))
    return -ll  #Сразу готовим для оптимизации

def like(q, x):
    n = len(x)
    l = np.power((2 / q), n) * np.prod(x) * np.prod(np.exp(-np.square(x) / q))
    return l

In [86]:
x1 = np.arange(1, 7, 0.001)
q_hat1 = optimize.minimize(loglike, 7, x1, method = 'Nelder-Mead')['x'][0]
q_hat1

18.995959472656246

Оставила ячейку выше как есть. Главная проблема (если так можно сказать) с которой я столкнулась - мы подаем на вход логарифмической функции правдоподобия значения, распределенные не с заданной в условии плотностью, а просто какие-то. Это не очень корректно. 

Исправим это. Для этого воспользуюсь функцией scipy.stats.rv_continuous, чтобы сгенерировать выборку с нужным распределением.

In [82]:
from scipy.stats import rv_continuous

class my_distr(rv_continuous):
    def _pdf(self, x, q=0.1):
        return ((2 * x) / q) * np.exp(-np.square(x) / q)
my_dist = my_distr(shapes='my_dist')


In [87]:
x1 = my_dist._pdf(x1)

In [88]:
q_hat = optimize.minimize(loglike, 1, x1, method = 'Nelder-Mead')['x'][0]
q_hat

3.5885014071368488e-09

In [89]:
def fisher(q, x):
    n = len(x)
    return (-n * q + 2 * n * np.exp(-np.square(x) /  q) * (q + np.square(x))) / (np.power(q, 3)) # E(-H)

Ifi_hat = fisher(q_hat1, x1)[0]
Ifi_hat

16.62756987886235

In [90]:
var_q_hat = 1 / Ifi_hat
var_q_hat

0.06014107938113321

In [91]:
left_bound = q_hat - 1.96 * np.sqrt(var_q_hat)
right_bound = q_hat + 1.96 * np.sqrt(var_q_hat)
print('q \in [%f; %f]' % (left_bound, right_bound))


q \in [-0.480664; 0.480664]


**ЗАДАНИЕ 4**

In [69]:
x = np.random.normal(0, 1, 300)  #генерируем 300 x_i
ln_u = np.random.normal(0, 1, 300)  #генерируем 300 ln(u_i)
u = np.exp(ln_u)  #получаем u_i путем взятия экспоненты от ln(u_i)
y = 3 * np.exp(x) * u

In [70]:
def num_ml(x, y):
    n = len(x)
    b2 = (np.sum(np.log(y)) * np.sum(x) - n * np.sum(x * np.log(y))) / (np.square(np.sum(x)) - n * np.sum(np.square(x)))
    b1 = np.exp((np.sum(np.log(y)) - b2 * np.sum(x)) / (n))
    return b1, b2

Пункт d:

In [71]:
num_ml(x, y)

(3.152979624998214, 1.0420012293056187)

Численно всегда выходит примерно 3 и 1, так что округлю до 3 и 1 при решении пункта e)

Пункт e:

In [72]:
def L(x, y, b1, b2):
    n = len(x)
    return n * np.log(1 / np.sqrt(2 * np.pi)) - 1/2 * np.sum(np.square(np.log(y) - np.log(b1) - b2 * x))

Запишем нашу гипотезу и ml оценку

In [76]:
b_0 = (1, 2)
b_ml = num_ml(x, y)

Почитаем $LR = 2(L(b_{H_0}) - L(b_{ML}))$

In [81]:
LR = 2 * (L(x, y, b_ml[0], b_ml[1]) - L(x, y, b_0[0], b_0[1]))
print('Значение LR =', LR)

Значение LR = 646.1119662470614


LR больше, чем Хи квадрат критическое $\approx$ 3, 96, значит $H_0$ не принимается.

Ура, мы справились!